# ------------------------------------------- First Submission -------------------------------------------
Neural Network with Backpropagation-Learning:

First reading the dataset and formatting (and imports):

In [ ]:
import random as random
from math import exp
from csv import reader

def readFile(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            dataset.append(row)

    for row in dataset[1:]:
        for index, datapoint in enumerate(row):
            if index < 4:
                row[index] = int(datapoint)
            if (index % 2) == 0 and index > 4 and index < 45:
                row[index] = float(datapoint)
    return dataset       
    
globalDataset = readFile("diterpene_shuf.csv")
globalDataset.remove(globalDataset[0])

Normalizing the frequencies:

In [ ]:
def normalize(dataset):
    frequencies = list()
    for column in dataset:
        for f in column:
            if type(f) == float:
                frequencies.append(f)
    minvalue = min(frequencies)
    maxvalue = max(frequencies)

    normalizedDataset = list(dataset)
    for row in normalizedDataset:
        for index, datapoint in enumerate(row):
            if (index % 2) == 0 and index > 4 and index < 45:
                row[index] = (datapoint - minvalue)/(maxvalue-minvalue)
    return normalizedDataset
                
globalDataset = normalize(globalDataset)

I define a function for creating a neural network with random initial weights: (I chose only one hidden layer)

In [ ]:
def createNetwork(numInputs, numHidden, numOutput):
    neuralNetwork = list()
    hiddenLayer = [{'weights':[random.random() for i in range(numInputs + 1)]} for i in range (numHidden)]
    outputLayer = [{'weights':[random.random() for i in range(numHidden + 1)]} for i in range (numOutput)]
    neuralNetwork.append(hiddenLayer)
    neuralNetwork.append(outputLayer)
    return neuralNetwork

I define functions for the activation function (sigmoid function), the handling of forward and backward propagation and updating the weight values:

In [ ]:
def activation(weights, input):
    bias = weights[-1]
    activationSum = bias
    for i in range(len(input) - 1):
        activationSum += weights[i] * input[i]
    return 1.0 / (1.0 + exp(activationSum * -1))

def forwardPropagate(neuralNetwork, data):
    input = data
    for layer in neuralNetwork:
        result = list()
        for node in layer:
            node['output'] = activation(node['weights'], input)
            result.append(node['output'])
        input = result
    return result

def backwardPropagate(neuralNetwork, expectedResult):
    for i in reversed(range(len(neuralNetwork))):
        layer = neuralNetwork[i]
        errors = list()
        if i == len(neuralNetwork)-1:
            for j in range(len(layer)):
                errors.append(layer[j]['output'] - expectedResult[j])
        else:
            for j in range(len(layer)):
                error = 0.0
                for node in neuralNetwork[i+1]:
                    error += node['weights'][j] * node['delta']
                errors.append(error)
        for j in range(len(layer)):
            node = layer[j]
            node['delta'] = errors[j] * (node['output'] * (1.0 - node['output']))
        
def updateWeights(neuralNetwork, data, learningRate):
    for i in range(len(neuralNetwork)):
        input = data[:-1]
        if i != 0:
            input = [node['output'] for node in neuralNetwork[i-1]]
        for node in neuralNetwork[i]:
            for j in range(len(input)):
                node['weights'][j] -= learningRate * node['delta'] * input[j]
            #bias
            node['weights'][-1] -= learningRate * node['delta']

My next step will be to define how to get the output of the algorithm:

In [ ]:
classes = list(set([data[-1] for data in globalDataset]))

def cleanOutput(nodeOutput):
    outputIndex = nodeOutput.index(max(nodeOutput))
    return classes[outputIndex]

To avoid overfitting I use 10-fold cross validation, so I create 10 roughly equal sized splits of the (normalized) dataset:
Also I remove any unused input from the datasetCopy

In [ ]:
def reformatDataset(oldDataset):
    datasetCopy = list(oldDataset)
    newDataset = list()
    for row in datasetCopy:
        newEntry = list()
        for index, data in enumerate(row):
            if index > 4 and index < len(row) - 1:
                if index % 2 != 0:
                    if data == 's':
                        newEntry.extend([row[index + 1], 0, 0, 0])
                    elif data == 'd':
                        newEntry.extend([0, row[index + 1], 0, 0])
                    elif data == 't':  
                        newEntry.extend([0, 0, row[index + 1], 0])
                    else:
                        newEntry.extend([0, 0, 0, row[index + 1]])
            if index == len(row) - 1:
                newEntry.append(data)
        newDataset.append(newEntry)
    return newDataset
  
def splitDataset(dataset, splits):
    datasetCopy = list(dataset)
    datasetSplit = list()
    for i in range(splits):
        splitSize = int(len(datasetCopy) / splits)
        split = list()
        while len(split) < splitSize:
            index = random.randint(0, len(datasetCopy) - 1)
            value = datasetCopy.pop(index)
            split.append(value)
        datasetSplit.append(split)
    return datasetSplit

Lastly I define methods for training the neural network with 9 of the splits and use the tenth for validation, in 10 seperate runs:

In [ ]:
def trainNetwork(neuralNetwork, trainset, learningRate, numEpoch):
    for epoch in range(numEpoch):
        for data in trainset:
            output = cleanOutput(forwardPropagate(neuralNetwork, data))
            expected = [0 for i in range(len(classes))]
            expected[classes.index(data[-1])] = 1
            backwardPropagate(neuralNetwork, expected)
            updateWeights(neuralNetwork, data, learningRate)

def testNetwork(neuralNetwork, testset):
    results = list()
    for data in testset:
        output = cleanOutput(forwardPropagate(neuralNetwork, data))
        results.append(output)
    return results

def runNetwork(dataset, learningRate, numEpoch, hiddenNodes):
    accuracy = list()
    for split in dataset:
        trainData = list(dataset)
        trainData.remove(split)
        trainData = sum(trainData, [])
        testData = list(split)
        network = createNetwork(len(split[0])-1, hiddenNodes, len(classes))
        trainNetwork(network, trainData, learningRate, numEpoch)
        results = testNetwork(network, testData)
        correct = 0
        for index, data in enumerate(testData):
            if results[index] == data[-1]:
                correct += 1
        successRate = correct / float(len(testData)) * 100
        print(successRate)

In [ ]:
learningRate = 0.5
numEpoch = 1200
hiddenNodes = 12
splitAmount = 10

datasetSplit = splitDataset(reformatDataset(globalDataset), splitAmount)

runNetwork(datasetSplit, learningRate, numEpoch, hiddenNodes)

The average accuracy with this configuration is between 45% and 50% which isn't great but it is somewhat working and I will try to further improve it for the second part of the assignment.

# ------------------------------------------- Improvements-------------------------------------------

To improve accuracy and be indepent of the order I want to try a few Ideas:\
First I combine the frequency and its multiplicity into a list:

In [ ]:
#must be done before reformatDataset
#also removes tha first 4 datapoints (expert data)
def combineDatapoints(dataset):
    newDataset = list()
    for row in dataset:
        newEntry = list()
        for index, data in enumerate(row):
            if index > 4 and index < len(row) - 1:
                if index % 2 != 0:
                    newEntry.append([row[index], row[index + 1]])
            if index == len(row) - 1:
                newEntry.append(data)
        newDataset.append(newEntry)
    return newDataset

Accuracy will be improved by sorting dataset by frequencies (ascending):

In [ ]:
#must be done before reformatDataset
def sortDataset(dataset):
    dataset = combineDatapoints(dataset)
    sortedDataset = list()
    for row in dataset:
        newEntry = list(row)
        classification = newEntry.pop()
        newEntry = sorted(newEntry, key = lambda x : (x[1], x[0]))
        newEntry.append(classification)
        sortedDataset.append(newEntry)
    return sortedDataset
    
def reformatCombined(dataset):
    datasetCopy = list(dataset)
    newDataset = list()
    for row in datasetCopy:
        newEntry = list()
        for index, data in enumerate(row):
            if index < len(row) - 1:
                if data[0] == 's':
                    newEntry.extend([data[1], 0, 0, 0])
                elif data[0] == 'd':
                    newEntry.extend([0, data[1], 0, 0])
                elif data[0] == 't':  
                    newEntry.extend([0, 0, data[1], 0])
                else:
                    newEntry.extend([0, 0, 0, data[1]])
            if index == len(row) - 1:
                newEntry.append(data)
        newDataset.append(newEntry)
    return newDataset

In [ ]:
learningRate = 0.5
numEpoch = 1200
hiddenNodes = 12
splitAmount = 10

sortedDataset = sortDataset(globalDataset)
sortedSplit = splitDataset(reformatCombined(sortedDataset), splitAmount)

runNetwork(sortedSplit, learningRate, numEpoch, hiddenNodes)

Another idea I wanted to try out was summing up the multiplicities and using the average of frequencies, this didn't really work out:

In [ ]:
#must be done before reformatDataset
def sumDataset(dataset):
    dataset = combineDatapoints(dataset)
    summedDataset = list()
    for row in dataset:
        newEntry = list()
        amountS = 0
        amountD = 0
        amountT = 0
        amountQ = 0
        sumS = 0.0
        sumD = 0.0
        sumT = 0.0
        sumQ = 0.0
        for index, data in enumerate(row):
            if index < len(row) - 1:
                if data[0] == 's':
                    amountS += 1
                    sumS += data[1]
                elif data[0] == 'd':
                    amountD += 1
                    sumD += data[1]
                elif data[0] == 't':  
                    amountT += 1
                    sumT += data[1]
                else:
                    amountQ += 1
                    sumQ += data[1]
            if index == len(row) - 1:
                avgS = sumS/amountS
                avgD = sumD/amountD
                avgT = sumT/amountT
                avgQ = sumQ/amountQ
                newEntry.extend([amountS, avgS, amountD, avgD, amountT, avgT, amountQ, avgQ])
                newEntry.append(data)
        summedDataset.append(newEntry)
    return summedDataset

In [ ]:
learningRate = 0.5
numEpoch = 1600
hiddenNodes = 8
splitAmount = 10

summedSplit = splitDataset(sumDataset(globalDataset), splitAmount)

runNetwork(summedSplit, learningRate, numEpoch, hiddenNodes)

I also wanted to look if sorting by multiplicities first makes a difference: \
It seems to be about 5% less accurate

In [ ]:
#must be done before reformatDataset
def sortByMultiplicities(dataset):
    dataset = combineDatapoints(dataset)
    mSortedDataset = list()
    for row in dataset:
        newEntry = list(row)
        classification = newEntry.pop()
        newEntry = sorted(newEntry, key = lambda x : (x[0], x[1]))
        newEntry.append(classification)
        mSortedDataset.append(newEntry)  
    return mSortedDataset

In [ ]:
learningRate = 0.5
numEpoch = 1200
hiddenNodes = 12
splitAmount = 10

mSortedSplit = splitDataset(reformatCombined(sortByMultiplicities(globalDataset)), splitAmount)

runNetwork(mSortedSplit, learningRate, numEpoch, hiddenNodes)

Last I wanted to see how effective randomizing the order after every training epoch is, but as expected it didn't show much improvement compared to doing nothing with the data at all

In [ ]:
def randomizedCopy(data):
    newData = list()
    for row in data:
        newData.append(randomize(row))
    return newData

def randomize(dataRow):
    newEntry = list()
    for index, data in enumerate(dataRow):
        if index < len(dataRow) - 1:
            if index % 2 == 0:
                newEntry.append([dataRow[index], dataRow[index + 1]])
    random.shuffle(newEntry)
    flatEntry = [item for sublist in newEntry for item in sublist]
    flatEntry.append(dataRow[-1])
    return flatEntry

def trainRandomizedNetwork(neuralNetwork, trainset, learningRate, numEpoch):
    for epoch in range(numEpoch):
        for data in trainset:
            output = cleanOutput(forwardPropagate(neuralNetwork, data))
            expected = [0 for i in range(len(classes))]
            expected[classes.index(data[-1])] = 1
            backwardPropagate(neuralNetwork, expected)
            updateWeights(neuralNetwork, data, learningRate)
        trainset = randomizedCopy(trainset)

learningRate = 0.5
numEpoch = 1200
hiddenNodes = 12
splitAmount = 10

datasetSplit = splitDataset(reformatDataset(globalDataset), splitAmount)

accuracy = list()
for split in datasetSplit:
    trainData = list(datasetSplit)
    trainData.remove(split)
    trainData = sum(trainData, [])
    testData = list(split)
    network = createNetwork(len(split[0])-1, hiddenNodes, len(classes))
    trainRandomizedNetwork(network, trainData, learningRate, numEpoch)
    results = testNetwork(network, testData)
    correct = 0
    for index, data in enumerate(testData):
        if results[index] == data[-1]:
            correct += 1
    successRate = correct / float(len(testData)) * 100
    print(successRate)

So the best method I tried out was simply sorting the dataset in order of frequencies, which resulted in around 80% accuracy